In [ ]:
# хочу идти по verified
# 1. строчка прайса
# 2. что верифицировано
# 3. что найдено

In [13]:
# from db.verified import report_row_num_un_uuid # (0)
# # UN skus for indexing
# from db.un import get_en_skus
# from db.company import get_addr_company_table
# from indexes.tn import build_tn_indexes, add_to_tn_index
# from indexes.skus import build_sku_row_idx

# from prices.helpers import load_pulse # (1)

# from zones.tn import tn_zone
# from zones.company import through_company_zone
# from indexes.company import build_company_idx

# Verified

In [2]:
### price_row_num, un_id
### OrderedDict([(39, 'a4fb182c-9d71-4d62-a405-70947f1dcc4e'),
###             (40, 'a4fb182c-9d71-4d62-a405-70947f1dcc4e'), ...])
verified = report_row_num_un_uuid() # (0)

# Price list

In [5]:
plist = load_pulse() # (1)

# Indexes

1. skus
2. trade_name

In [30]:
# # %time un_skus = get_en_skus()

# sku_indexes = build_sku_row_idx()
# # skus_rows_id_idx = sku_indexes['rows_id'] # {..., [`${row_num}`]: un_id, ...}
# skus_id_rows_idx = sku_indexes['rows_id_inv'] # {..., [`${un_id}`]: row_num, ...}

# %time tn_indexes = build_tn_indexes()

#
# Company
#
# from indexes.company import build_company_idx
# TASK!: address_id => company_id => company short name
# company_idx = build_company_idx()

#
# Company Address
#

# from db.company import get_addr_company_table
# addr_company_idx = get_addr_company_table()

# company_id_by_address_id = addr_company_idx['addr_cmp']

dict_keys(['recs', 'addr_cmp', 'cmp_addreses'])

In [41]:
# company_idx.keys()
# company_id_idx = company_idx['company_id_idx']
# company_src = company_idx['companies_table']
# company_src[:10]


# dosage

In [46]:
# from db.dosage import fetch_dosage_forms
# dosage_forms_idx['recs'] = fetch_dosage_forms()
dosage_forms_recs[0]

('0006df9c-f765-4821-82f0-0f5dd6b28126', 'лиофилизат спрея', '13.3')

In [54]:
# from indexes.forms import build_dosage_forms_idx
# dosage_forms_idx = build_dosage_forms_idx()
# dosage_row_id = dosage_forms_idx['id_row']
# dosage_forms_recs = dosage_forms_idx['recs']
# dosage_forms_idx.keys()
row, un_id = verified_items[0]
sku_row = skus_id_rows_idx[un_id]
un_sku_data = un_skus[sku_row]

sku_data = un_sku_data[1]
dosage_id_list = get_dosage_form_id(sku_data)
for dosage_id in dosage_id_list:
    dosage_row = dosage_row_id[dosage_id]
    dosage_rec = dosage_forms_recs[dosage_row]
    dosage_name = dosage_rec[1]
    print(dosage_name)

dosage_name

таблетки, покрытые оболочкой


'таблетки, покрытые оболочкой'

In [60]:
from helpers.skus import get_tn_from_data # ,  get_tn
from indexes.company import get_address_id
from indexes.forms import get_dosage_form_id

def show_sku(sku_rec: tuple):
    sku_id, sku_data = sku_rec
    tn = get_tn_from_data(sku_data)
    address_id = get_address_id(sku_data)
    # TODO:
    #     company_short_name_by_address(address_id)
    company_id = company_id_by_address_id[address_id]
    c_row = company_id_idx[company_id]
    c_rec = company_src[c_row]
    short_rus_name = c_rec[3] or c_rec[2] or c_rec[1]

    #
    # Dosage
    #
    dosage_id_list = get_dosage_form_id(sku_data)
    dosage_names = list()
    for dosage_id in dosage_id_list:
        dosage_row = dosage_row_id[dosage_id]
        dosage_rec = dosage_forms_recs[dosage_row]
        dosage_name = dosage_rec[1]
        dosage_names.append(dosage_name)
        
#     lform = get_
#     lform, dosage, count, pack, company = ('конц-т д/приг. р-ра инфуз.', ' 250 мг 5 мл.', 'фл.', 'х1', 'company')
#     msg = '{} {} {} {} {}'.format(tn, lform, dosage, pack, count, company)
    return '[{}] [{}] [{}]'.format(tn, ';'.join(dosage_names), short_rus_name)

In [63]:
verified_items = list(verified.items())[:10]
for i, (price_i, un_id) in enumerate(verified_items):
    # price
    sel_plist = plist[price_i - 1:price_i]
    sel_cols = sel_plist[sel_plist.columns[:3]]
    values = sel_cols.values[0]
    code, name, company = values
    print('[{}]\nP: {} [{}]'.format(i, name, company))
    # varified
    # un_id => str
    sku_row = skus_id_rows_idx[un_id]
    un_sku_data = un_skus[sku_row]
    sku_msg = show_sku(un_sku_data)
    print('U: {}'.format(sku_msg))
    print('S: {} {} {}'.format('tn_zone', 'dosage_zone', 'company_zone'))


[0]
P: 5-Нок табл. 50 мг х50 [Lek – Вектор]
U: [5-нок] [таблетки, покрытые оболочкой] [ЛЕК]
S: tn_zone dosage_zone company_zone
[1]
P: 5-Нок табл. 50 мг х50 [Lek – Сотекс]
U: [5-нок] [таблетки, покрытые оболочкой] [ЛЕК]
S: tn_zone dosage_zone company_zone
[2]
P: 5-Нок табл. п/о 50 мг х50 [Lek]
U: [5-нок] [таблетки, покрытые оболочкой] [ЛЕК]
S: tn_zone dosage_zone company_zone
[3]
P: 5-фторурацил конц-т д/приг. р-ра инфуз. 250 мг фл. 5 мл. х1 [Ebewe]
U: [5-фторурацил-эбеве] [концентрат для приготовления раствора для инфузий] [ЕБЕВЕ ФАРМА]
S: tn_zone dosage_zone company_zone
[4]
P: 5-фторурацил конц-т д/приг. р-ра инфуз. 250 мг фл. 5 мл. х5 [Ebewe]
U: [5-фторурацил-эбеве] [концентрат для приготовления раствора для инфузий] [ЕБЕВЕ ФАРМА]
S: tn_zone dosage_zone company_zone
[5]
P: 5-фторурацил конц-т д/приг. р-ра инфуз. 50 мг/мл фл. 20 мл. х1 не исп [Ebewe Pharma GesmbH Nfg KG]
U: [5-фторурацил-эбеве] [концентрат для приготовления раствора для инфузий] [ЕБЕВЕ ФАРМА]
S: tn_zone dosage_zone 

In [28]:
get_tn(60249)

'5-нок'

In [38]:
name_bow = get_bow(name)
name_bow

['5-нок', 'табл.', '50', 'мг', 'х50']

In [41]:
term0 = name_bow[0]

In [59]:
def term_docs(term='нок', terms_docs=terms_docs):
    if term not in terms_docs:
        return print('[-]')
    docs = terms_docs[term]
    print(len(docs), len(set(docs)))
    for doc in docs:
        print(get_tn(doc))

In [ ]:
indexes = build_tn_indexes()
# в верефицированных
rows_id_inv = indexes['rows_id_inv']
rows_id_inv['a4fb182c-9d71-4d62-a405-70947f1dcc4e']
terms_docs = indexes['terms_docs']
indexes.keys()

# END TRADE_NAME

[?] как добавить в индекс add_to_tn_index(term='5', docs=[60249])

In [71]:
def add_to_tn_index(term, doc):
    # get existing item or new list
    term_docs = term in terms_docs and terms_docs[term] or list()
    if doc in term_docs:
        return print('there is a doc[{}] in term[{}]'.format(doc, term))
    # can optimizate, where insert?
    term_docs.append(doc)
    newdocs = sorted(term_docs)
    terms_docs[term] = newdocs

# TN RULES

In [72]:
term = '5'
add_to_tn_index(term, doc=60249)
term = 'нок'
add_to_tn_index(term, doc=60249)

there is a doc[60249] in term[5]
there is a doc[60249] in term[нок]


In [60]:
term_docs()

[-]


In [74]:
tn_zone(name, terms_docs)

[[('5-нок', [60249]),
  ('табл.', None),
  ('50',
   [9203,
    24099,
    24104,
    29817,
    50342,
    50343,
    50344,
    91156,
    106090,
    131915,
    131916,
    132005]),
  ('мг', [68457]),
  ('х50', None)]]

# Company

In [128]:
# company_idx = build_company_idx()
# company_terms_docs = company_idx['terms_docs']
# company_id_idx_inv = company_idx['company_id_idx_inv']
# companies_table = company_idx['companies_table']

# un addresses 

addr_company_idx = get_addr_company_table()
addr_cmp_idx = addr_company_idx['addr_cmp'] #
cmp_addr_idx = addr_company_idx['cmp_addreses']
print(addr_company_idx.keys())

company_idx.keys()

dict_keys(['recs', 'addr_cmp', 'cmp_addreses'])


dict_keys(['company_id_idx', 'company_id_idx_inv', 'terms_docs', 'companies_table', 'companies_synonyms'])

In [137]:
company_id_idx = company_idx['company_id_idx']
company_id_idx

In [5]:
# sel_plist_3_fields = sel_plist[sel_plist.columns[:3]].values
# company_zone = company_index_to_docs(through_company_zone(sel_plist_3_fields, company_terms_docs))

In [95]:
def company_index_to_docs(res):
    print(res)

In [ ]:
# row_num -> company_id
# TODO: row_num -> docs
company_row = 2696
company_id = company_id_idx_inv[company_row]
print('company_id {}'.format(company_id))
addresses_cmp_id = cmp_addr_idx[addr_cmp_idx['64cb8085-926e-414a-90cf-d38a1dc4ef0b']]
print('company_id address list: {}'.format(addresses_cmp_id))
print('TODO: list of skus:')
# print('company_id {}'.format(company_id))
# # print(companies_table[company_row])
# cmp_addr_idx